In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import requests

# This CATALOG_URL and MANAGEMENT_URL work for the "docker compose" testing and development environment.
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8080/catalog"
MANAGEMENT_URL = "http://server:8080/management"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.7.0"

# Create a new Warehouse

In [2]:
response = requests.post(f"{MANAGEMENT_URL}/v1/warehouse",
              json={
                "warehouse-name": "warehouse_1",
                "storage-profile": {
                    "type": "s3",
                    "bucket": "examples",
                    "key-prefix": "path/to/warehouse_1",
                    "assume-role-arn": None,
                    "endpoint": "http://minio:9000",
                    "region": "local-01",
                    "path-style-access": True,
                    "sts-enabled": False
                },
                # Storage Credentials for the profile specified above.
                # These credentials are used to grant clients access to specific files in the storage.
                # Clients do not need to know those credentials and will never obtain them directly.
                "storage-credential": {
                    "type": "s3",
                    "credential-type": "access-key",
                    "aws-access-key-id": "minio-root-user",
                    "aws-secret-access-key": "minio-root-password"
                }
            })
print(f"{response.status_code}: {response.reason}")

201: Created


In [3]:
response = requests.post(f"{MANAGEMENT_URL}/v1/warehouse",
              json={
                # Name of the new warehouse
                "warehouse-name": "warehouse_2",
                # Physical location of this warehouse
                "storage-profile": {
                    "type": "s3",
                    "bucket": "examples",
                    # you can change the prefix to something else, f. ex. f"{WAREHOUSE}
                    # as long as it is unique in the bucket
                    "key-prefix": "path/to/new-warehouse_1/",
                    "assume-role-arn": None,
                    "endpoint": "http://minio:9000",
                    "region": "local-01",
                    "path-style-access": True,
                    "sts-enabled": False
                },
                # Storage Credentials for the profile specified above.
                # These credentials are used to grant clients access to specific files in the storage.
                # Clients do not need to know those credentials and will never obtain them directly.
                "storage-credential": {
                    "type": "s3",
                    "credential-type": "access-key",
                    "aws-access-key-id": "minio-root-user",
                    "aws-secret-access-key": "minio-root-password"
                }
            })
print(f"{response.status_code}: {response.reason}")

201: Created


In [4]:
response = requests.post(f"{MANAGEMENT_URL}/v1/warehouse",
              json={
                "warehouse-name": "warehouse_3",
                "storage-profile": {
                    "type": "s3",
                    "bucket": "examples",
                    "key-prefix": "path/to/warehouse_3",
                    "sts-role-arn": "minio-root-user",
                    "endpoint": "http://minio:9000",
                    "region": "local-01",
                    "path-style-access": True,
                    "sts-enabled": True
                },
                # Storage Credentials for the profile specified above.
                # These credentials are used to grant clients access to specific files in the storage.
                # Clients do not need to know those credentials and will never obtain them directly.
                "storage-credential": {
                    "type": "s3",
                    "credential-type": "access-key",
                    "aws-access-key-id": "minio-root-user",
                    "aws-secret-access-key": "minio-root-password"
                }
            })
print(f"{response.status_code}: {response.reason}")

201: Created


In [5]:
response = requests.post(f"{MANAGEMENT_URL}/v1/warehouse",
              json={
                "warehouse-name": "warehouse_4",
                "storage-profile": {
                    "type": "s3",
                    "bucket": "examples",
                    "key-prefix": "path/to/warehouse_4",
                    "sts-role-arn": "minio-root-user",
                    "endpoint": "http://minio:9000",
                    "region": "local-01",
                    "path-style-access": True,
                    "sts-enabled": True
                },
                # Storage Credentials for the profile specified above.
                # These credentials are used to grant clients access to specific files in the storage.
                # Clients do not need to know those credentials and will never obtain them directly.
                "storage-credential": {
                    "type": "s3",
                    "credential-type": "access-key",
                    "aws-access-key-id": "minio-root-user",
                    "aws-secret-access-key": "minio-root-password"
                }
            })
print(f"{response.status_code}: {response.reason}")

201: Created


In [13]:
requests.get(f"{CATALOG_URL}/v1/config?warehouse=warehouse_3").json()

{'overrides': {'prefix': '059b7a06-a791-11ef-9703-8f76f1867c0b',
  'uri': 'http://server:8080/catalog'},
 'defaults': {'s3.delete-enabled': 'false'}}

In [11]:
requests.get(f"{CATALOG_URL}/v1/ef725de2-a78d-11ef-b433-3bd381e7ba96/namespaces/my_namespace/tables").json()

{'error': {'message': 'Namespace action can_list_tables forbidden',
  'type': 'NamespaceActionForbidden',
  'code': 403,
  'stack': ['01934bca-ac5d-77a2-b6b1-56baa6e59125']}}

# Connect with Spark

In [8]:
config = {
    f"spark.sql.catalog.catalog1": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.catalog1.type": "rest",
    f"spark.sql.catalog.catalog1.uri": CATALOG_URL,
    f"spark.sql.catalog.catalog1.warehouse": "warehouse_1",
    f"spark.sql.catalog.catalog1.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    f"spark.sql.catalog.catalog1.s3.remote-signing-enabled": "false",
    f"spark.sql.catalog.catalog2": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.catalog2.type": "rest",
    f"spark.sql.catalog.catalog2.uri": CATALOG_URL,
    f"spark.sql.catalog.catalog2.warehouse": "warehouse_2",
    f"spark.sql.catalog.catalog2.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    f"spark.sql.catalog.catalog3": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.catalog3.type": "rest",
    f"spark.sql.catalog.catalog3.uri": CATALOG_URL,
    f"spark.sql.catalog.catalog3.warehouse": "warehouse_3",
    f"spark.sql.catalog.catalog3.s3.remote-signing-enabled": "false",
    f"spark.sql.catalog.catalog3.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    f"spark.sql.catalog.catalog4": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.catalog4.s3.remote-signing-enabled": "false",
    f"spark.sql.catalog.catalog4.type": "rest",
    f"spark.sql.catalog.catalog4.uri": CATALOG_URL,
    f"spark.sql.catalog.catalog4.warehouse": "warehouse_4",
    f"spark.sql.catalog.catalog4.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
}

In [9]:
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

data = pd.DataFrame([[1, 'a-string', 2.2]], columns=['id', 'strings', 'floats'])
sdf = spark.createDataFrame(data)

In [7]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS catalog1.my_namespace")
sdf.writeTo("catalog1.my_namespace.my_table").createOrReplace()

Py4JJavaError: An error occurred while calling o100.createOrReplace.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (c4d822b796af executor driver): org.apache.iceberg.exceptions.BadRequestException: Malformed request: No prefix specified. The warehouse-id must be provided as prefix in the URL.
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:208)
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:188)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:211)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:323)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:379)
	at org.apache.iceberg.rest.RESTClient.post(RESTClient.java:122)
	at org.apache.iceberg.aws.s3.signer.S3V4RestSignerClient.sign(S3V4RestSignerClient.java:341)
	at software.amazon.awssdk.core.internal

In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS catalog2.my_namespace")
sdf.writeTo("catalog2.my_namespace.my_table").createOrReplace()

In [10]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS catalog3.my_namespace")
sdf.writeTo("catalog3.my_namespace.my_table").createOrReplace()

Py4JJavaError: An error occurred while calling o104.createOrReplace.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (51276b3225ba executor driver): org.apache.iceberg.exceptions.BadRequestException: Malformed request: No prefix specified. The warehouse-id must be provided as prefix in the URL.
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:208)
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:188)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:211)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:323)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:379)
	at org.apache.iceberg.rest.RESTClient.post(RESTClient.java:122)
	at org.apache.iceberg.aws.s3.signer.S3V4RestSignerClient.sign(S3V4RestSignerClient.java:341)
	at software.amazon.awssdk.core.internal

In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS catalog4.my_namespace")
sdf.writeTo("catalog4.my_namespace.my_table").createOrReplace()